# NMEG SPEI

Calculate SPEI (Standardized precipitation-evaporation index) for NMEG data and historical SPEI at the sites.

## First calculate SPEI for NMEG data period

In [1]:
# Load daily data files
source('../r_functions/load_nmeg.r')

seg <- daily_to_xts(get_daily_file('Seg', 'aflx', make_new=FALSE))
ses <- daily_to_xts(get_daily_file('Ses', 'aflx', make_new=FALSE))
sen <- daily_to_xts(get_daily_file('Sen', 'aflx', make_new=FALSE))
# wjs has some empty rows that are a problem
wjs <- daily_to_xts(get_daily_file('Wjs', 'aflx', make_new=FALSE))
nonemptyrows <- rowSums(is.na(wjs)) != ncol(wjs)
firstnonempty <- min(which(nonemptyrows))
wjs <- wjs[firstnonempty:nrow(wjs),]

mpj <- daily_to_xts(get_daily_file('Mpj', 'aflx', make_new=FALSE))
mpg <- daily_to_xts(get_daily_file('Mpg', 'aflx', make_new=FALSE))
vcp <- daily_to_xts(get_daily_file('Vcp', 'aflx', make_new=FALSE))
vcm <- daily_to_xts(get_daily_file('Vcm', 'aflx', make_new=FALSE))

head(wjs)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



           GPP_g_int RECO_g_int FC_F_g_int ET_mm_24hint_0 P_F_sum  TA_F_avg
2007-05-04 0.9376559   1.353289  0.4156327       1.036631     0.0 19.040114
2007-05-05 4.3097359   2.945274 -1.3644615       2.043919     0.0 11.717296
2007-05-06 4.5503009   2.793961 -1.7563396       1.594291     0.0  8.113653
2007-05-07 3.6014284   2.880228 -0.7212009       1.569668     0.0 10.484494
2007-05-08 3.0864031   2.901163 -0.1852404       1.918436     5.8  9.967429
2007-05-09 3.8685696   2.950288 -0.9182811       1.788734     0.0 11.999762
           RH_F_avg SW_IN_F_avg RNET_F_avg VPD_F_avg PAR_avg TA_F_min VPD_F_min
2007-05-04 20.36750    246.6755         NA 1.7772101      NA 15.37122  1.248726
2007-05-05 38.09231    365.9140         NA 0.8719900      NA  6.75012  0.371017
2007-05-06 51.02854    365.9140         NA 0.5806247      NA  4.19226  0.197185
2007-05-07 46.78256    388.7169         NA 0.8043145      NA  2.86882  0.156562
2007-05-08 77.36767    337.2944         NA 0.2936560      NA  7.1134

## Examine climatic water deficit for all sites

In [8]:
# Calculate climatic water deficit
sitelist <- list( seg, ses, sen, wjs, mpj, mpg, vcp, vcm )
sitenames <- list( 'Seg', 'Ses', 'Sen', 'Wjs', 'Mpj', 'Mpg', 'Vcp', 'Vcm' )
cwdiff <- data.frame()
for (i in 1:8){
    cwdiff <- cbind( cwdiff, apply.monthly(sitelist[[i]]$P_F, FUN=sum) - 
                    apply.monthly(sitelist[[i]]$PET_mm_dayint, FUN=sum))
}
colnames(cwdiff) <- sitenames
head(cwdiff)
#plot.xts(cwdiff, screens=1)


                  Seg        Ses Sen       Wjs Mpj Mpg         Vcp         Vcm
2007-01-31  -4.355087  -3.224694  NA        NA  NA  NA   13.654987   63.681267
2007-02-28 -13.779283 -11.052548  NA        NA  NA  NA    7.133503   24.946473
2007-03-31 -11.541623  -9.908601  NA        NA  NA  NA   26.366218   14.449853
2007-04-30 -33.962316 -22.912343  NA        NA  NA  NA  -30.533337    3.894350
2007-05-31 -64.449426 -35.402545  NA -65.22351  NA  NA  -33.404282    2.293258
2007-06-30 -59.818992 -65.363961  NA -33.36749  NA  NA -131.978907 -100.864471

## Load packages for calculating SPEI
This package is on CRAN [here](https://cran.r-project.org/web/packages/SPEI/index.html). Website for the project is <http://sac.csic.es/spei/index.html>

If not installed use "install.packages('SPEI')"

In [9]:
library('SPEI')
library('xts')

Loading required package: lmomco
Loading required package: parallel
# Package SPEI (1.6) loaded [try SPEINews()].


In [11]:
# Function for retrieving SPEI values from a dataframe at
# a number of different timesteps
get_spei_steps <- function(df, sitename, tstep='monthly',
                           int_steps=seq(1, 12), plot=TRUE){
    if (tstep=='monthly'){
        freq <- 12
        } else if (tstep=='weekly'){
        freq <- 52
    }
    # Get start date
    startmon <- as.yearmon(index(df[1]))
    startyr <- floor(as.numeric(startmon))
    startmon <- as.numeric( format( startmon, '%m'))
    # Calculate climatic water difference on a weekly or monthly scale
    if (tstep=='monthly'){
        cwdiff <- apply.monthly(df$P_F_sum, FUN=sum) - apply.monthly(df$PET_mm_dayint, FUN=sum)
    } else if (tstep=='weekly'){
        cwdiff <- apply.weekly(df$P_F_sum, FUN=sum) - apply.weekly(df$PET_mm_dayint, FUN=sum)
    }
    colnames(cwdiff) <- sitename
    for (i in 1:length(int_steps)){
        # Integration period for SPEI (in number of timesteps)
        SPEI_int_per <- int_steps[i]
        # Get spei for that integration period
        spei_int <- spei(ts(cwdiff, frequency=freq, start=c(startyr, startmon)), SPEI_int_per, na.rm=TRUE)
        if (plot){
            plot(spei_int)
        }
        # Extract the spei values from the returned object and make xts
        spei_int <- xts(as.vector(spei_int$fitted),  index(cwdiff))
        colnames(spei_int) <- paste('SPEI_', tstep, '_', as.character(SPEI_int_per), sep='')
        if (i==1){
            df_new <- spei_int
        } else if (i > 1) {
            df_new <- cbind(df_new, spei_int)
        }
    }
    print(head(df_new))
    
    # Somehow a funky entry at 2009-2-12 is created, remove
    #print(nrow(df_new))
    #df_new <- df_new[c('::2009-01-31', '2009-02-28::')]
    #print(nrow(df_new))
    
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(df_new)))
    df_new[!is.finite(df_new)] <- NA
    print(sum(is.na(df_new)))
    
    # Interpolate over NA values
    df_new_interp <- na.approx(df_new)
        
    return( list(df_new, df_new_interp) )
}

In [59]:
sitenum <- 3
int_steps1 <- seq(1, 12)
print( sitenames[[sitenum]] )
print( head(sitelist[[sitenum]]) )
#df <- sitelist[[sitenum]]
#cwdiff1 <- apply.weekly(df$P_F, FUN=sum) - apply.weekly(df$PET_mm_dayint, FUN=sum)
#spei_int1 <- spei(ts(cwdiff1, frequency=52, start=c(startyr, 1)), SPEI_int_per, na.rm=TRUE)
spei_weekly <- get_spei_steps(sitelist[[sitenum]], sitenames[[sitenum]], int_steps=int_steps1, tstep='monthly', plot=FALSE)
#print( spei_weekly[[2]] )

[1] "Sen"
           GPP_g_int RECO_g_int FC_F_g_int ET_mm_24hint_0 P_F_sum   TA_F_avg
2010-01-01 0.1136120  0.4259083 0.31229628      0.4032358       0 -2.5114543
2010-01-02 0.2161500  0.3143947 0.09824470      0.2736742       0 -0.2682929
2010-01-03 0.1204642  0.3600118 0.23954760      0.2097532       0 -0.9624908
2010-01-04 0.1580211  0.3750380 0.21701696      0.1573903       0 -3.0812138
2010-01-05 0.1367253  0.3438134 0.20708806      0.1526637       0 -3.3661277
2010-01-06 0.1897916  0.2673470 0.07755541      0.1364428       0 -0.6928112
           RH_F_avg SW_IN_F_avg RNET_F_avg VPD_F_avg  TA_F_min VPD_F_min
2010-01-01 59.81988    166.7249   45.91650 0.2537424 -10.21888  0.037451
2010-01-02 62.02385    146.0594   36.79920 0.2731999  -6.29779  0.035677
2010-01-03 61.37402    165.1616   37.36449 0.2668355  -7.10839  0.037133
2010-01-04 66.35104    164.8700   40.86090 0.2208267  -9.35950  0.027159
2010-01-05 67.14276    161.6243   40.18154 0.2360786 -10.76979  0.020663
2010-01-06 60

In [56]:
# Create monthly files

for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    sitenm <- sitenames[[j]]
    int_steps1 <- seq(1, 24)
    if(sitenm=='Sen'){
        int_steps1 <- seq(1, 23)# For some reason Sen has a problem with 24 month SPEI
    }
    spei_monthly <- get_spei_steps(df_j, sitenm, int_steps=int_steps1, tstep='monthly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_monthly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_monthly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_monthly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_monthly_1 SPEI_monthly_2 SPEI_monthly_3 SPEI_monthly_4
2010-01-31             NA             NA             NA             NA
2010-02-28      1.6160337       1.597385             NA             NA
2010-03-31      1.3332477             NA       1.595097             NA
2010-04-30      0.9209119       1.176840       1.439131      1.5296005
2010-05-31     -1.3410072      -1.422840      -0.882110      0.4901951
2010-06-30     -0.9326078      -1.373149      -1.558114     -1.2083621
           SPEI_monthly_5 SPEI_monthly_6 SPEI_monthly_7 SPEI_monthly_8
2010-01-31             NA             NA             NA             NA
2010-02-28             NA             NA             NA             NA
2010-03-31             NA             NA             NA             NA
2010-04-30             NA             NA             NA             NA
2010-05-31      1.1641243             NA             NA             NA
2010-06-30      0.2263833       0.989293             NA             NA
      

In [55]:
# Now weekly
#
# WARNING - note that weekly SPEI generates more NA values - not sure why
#
int_steps2 <- seq(4, 96, 4)
for (j in 1:length(sitelist)){
    df_j <- sitelist[[j]]
    spei_weekly <- get_spei_steps(df_j, 'Ses', int_steps=int_steps2, tstep='weekly', plot=FALSE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-',
                     sitenames[[j]], '.csv', sep='')
    write.zoo(spei_weekly[[1]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    outfile <- paste('../processed_data/spei/SPEI_weekly_US-', sitenames[[j]],
                     '_nainterp.csv', sep='')
    write.zoo(spei_weekly[[2]], file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

           SPEI_weekly_4 SPEI_weekly_8 SPEI_weekly_12 SPEI_weekly_16
2010-01-03            NA            NA             NA             NA
2010-01-10            NA            NA             NA             NA
2010-01-17            NA            NA             NA             NA
2010-01-24            NA            NA             NA             NA
2010-01-31      1.408190            NA             NA             NA
2010-02-07      1.785578            NA             NA             NA
           SPEI_weekly_20 SPEI_weekly_24 SPEI_weekly_28 SPEI_weekly_32
2010-01-03             NA             NA             NA             NA
2010-01-10             NA             NA             NA             NA
2010-01-17             NA             NA             NA             NA
2010-01-24             NA             NA             NA             NA
2010-01-31             NA             NA             NA             NA
2010-02-07             NA             NA             NA             NA
           SPEI_week

## Now calculate SPEI for historical periods (PRISM)

In [4]:
fpath <- '/home/greg/sftp/eddyflux/Ancillary_met_data/PRISM_monthly/'
fname_ppt <- 'PRISM_Monthly_ppt_1981_2015.csv'
fname_tmean <-'PRISM_Monthly_tmean_1981_2015.csv'
#header <- read.csv(fname, skip=3, nrows=1)
df_ppt <- read.csv(paste(fpath, fname_ppt, sep=''))
df_tmean <- read.csv(paste(fpath, fname_tmean, sep=''))
df_ppt <- xts( df_ppt[,2:ncol(df_ppt)], as.Date(df_ppt$date))
df_tmean <- xts( df_tmean[,2:ncol(df_tmean)], as.Date(df_tmean$date))
head(df_ppt)
head(df_tmean)

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp
1981-01-31   3.88   5.38   5.26   5.38   6.10   5.58   4.21   4.39
1981-02-28   5.97   8.40   8.55   4.73   4.95   4.59  28.38  23.72
1981-03-31  16.98  24.01  24.16  16.08  16.50  16.11 161.74 136.64
1981-04-30  11.00  19.80  19.94  10.30  13.20  10.74  47.23  43.04
1981-05-31  10.12  16.72  15.86   9.75  10.13   9.26  88.62  76.18
1981-06-30  20.76  16.05  15.49  12.04  12.21  10.90  61.35  57.34

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp
1981-01-31  2.075  1.340  1.515  3.615  3.535  3.620 -2.020 -0.890
1981-02-28  4.055  3.510  3.665  6.205  6.010  6.320 -2.580 -1.385
1981-03-31  5.245  4.555  4.675  8.080  7.910  8.095 -2.415 -1.115
1981-04-30 12.065 11.015 11.170 15.190 14.915 15.240  3.945  5.420
1981-05-31 14.975 13.525 13.695 18.585 18.305 18.615  6.125  7.640
1981-06-30 21.625 20.955 21.180 24.770 24.505 24.850 13.450 14.710

           US.Wjs US.Mpj US.Mpg US.Seg US.Sen US.Ses US.Vcm US.Vcp
2015-07-31 21.619 21.099 21.319 24.105 23.864 24.324 13.134 14.850
2015-08-31 21.820 21.394 21.585 24.864 24.605 25.034 13.629 15.339
2015-09-30 19.835 19.315 19.484 22.249 21.999 22.444 12.124 13.784
2015-10-31 13.130 12.669 12.809 15.679 15.454 15.849  7.074  8.325
2015-11-30  5.549  4.830  4.985  6.745  6.670  6.879 -0.380  0.825
2015-12-31  1.445  1.005  1.250  1.644  1.590  1.875 -4.159 -3.084

In [5]:
# Get site coordinates
coords <- read.csv('../site_coords.txt')
coords

,sitecode,lat,lon
1,US-Wjs,34.42549,-105.8615
2,US-Mpj,34.43845,-106.2377
3,US-Mpg,34.44682,-106.2134
4,US-Seg,34.36233,-106.7019
5,US-Sen,34.35802,-106.6799
6,US-Ses,34.33494,-106.7442
7,US-Vcm,35.88845,-106.5321
8,US-Vcp,35.86423,-106.5967


## Calculate SPEI using thornwaite PET

In [10]:
plot_spei <- FALSE
int_steps1 <- seq(1, 24)

for (i in 1:length(sitenames)){
    # Get site name and latitude
    sitename1 <- paste('US-', sitenames[[i]], sep='')
    lat <- coords[coords$sitecode==sitename1, 2]
    sitename2 <- paste('US.', sitenames[[i]], sep='') # Format sitename for PRISM datasets
    # Calculate pet and climatic water diff with thornthwaite (convert to ts for this)
    ta <- ts(df_tmean[,sitename2], frequency=12)
    ppt <- ts(df_ppt[,sitename2], frequency=12)
    pet <- thornthwaite(ta, lat, na.rm=T)
    cwdiff <- ppt - pet
    for (j in 1:length(int_steps1)) {
        int_per <- int_steps1[j]
        spei_int <- spei(ts(cwdiff, frequency=12, start=c(1981, 1)), int_per, na.rm=TRUE)
        if (plot_spei){
            plot(spei_int)
        }
        spei_int <- xts(as.vector(spei_int$fitted),  index(df_tmean))
        colnames(spei_int) <- paste('SPEI_monthly_', as.character(int_per), sep='')
        if (j==1){
            spei_site <- spei_int
        } else {
            spei_site <- cbind(spei_site, spei_int)
        }
    }
    # There may be both infinite and NA values in the output
    # Convert -Inf to NA
    print(sum(is.na(spei_site)))
    spei_site[!is.finite(spei_site)] <- NA
    print(sum(is.na(spei_site)))
    # Actually there are no NA values - ignore this
    
    # Interpolate over NA values
    #spei_site_interp <- na.approx(spei_site)
    
    # Write files
    outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
                     '.csv', sep='')
    write.zoo(spei_site, file = outfile,
              index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
    
    #outfile <- paste('../processed_data/spei/SPEI_PRISM_monthly_', sitename1,
    #                 '_nainterp.csv', sep='')
    #write.zoo(spei_site_interp, file = outfile,
    #          index.name = "Date", sep=',', row.names = FALSE, col.names=TRUE)
}

[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
[1] 276
